In [15]:
import os
import pandas as pd
os.environ['CUDA_VISIBLE_DEVICES'] = '2' # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!

from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
)

from asset.tokenization_kobert import KoBertTokenizer
from asset import tokenization_kobert
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [16]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [17]:
model = EncoderDecoderModel.from_pretrained('dacon-v0/checkpoint-15500')
model.eval()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

In [18]:
def read_input(path):
    test_question = []
    test = pd.read_csv(path)
    for ut in test['질문']:
        test_question.append(ut)

    return test_question

In [19]:
test_q= read_input('./bigdata/test.csv')

In [20]:
def translate(text):
    embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    embeddings = {k: v for k, v in embeddings.items()}
    # generate 함수에 하이퍼파라미터 추가
    output_sequences = model.generate(
        **embeddings,
        max_length=300,
        temperature=0.9,
        top_k=1,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )
    
    tmp = trg_tokenizer.decode(output_sequences[0, 1:-1].cpu())
    return tmp

In [21]:
print("size of full : ",len(test_q))

size of full :  130


In [ ]:
preds = []
for i in range(len(test_q)):
    sr_text = test_q[i] # dialect
    tg_text = translate(sr_text) # translate
    preds.append(tg_text)
    if i<3:
        print("source_text : ",sr_text)
        print("result :      ",tg_text)
        print()
        print()

/home/leadawon5/gitfiles/venvs/myvenv/lib/python3.7/site-packages/transformers/generation/utils.py:1260: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"


source_text :  방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
result :       방청 페인트의 단점은 주로 시공 과정에서의 문제와 유지보수 측면에서 발생합니다. 제대로된 시공이 이뤄지지 않을 경우 균열이 발생하거나 오염이 쉽게 생길 수 있으며, 유지보수에 대한 노력과 비용이 필요할 수 있습니다. 또한, 특히 철재나 목재와 같은 소재에 적용될 경우에는 추가적인 관리가 필요할 수 있습니다. 이러한 점들을 고려하여 전문가의 조언을 구하는 것이 좋습니다. 때때로 방청 페인트를 선택할 때는 시공과 유지보수에 대한 신중한 고려가 필요합니다. 올바른 선택을 통해 방청 효과를 낼 수 있으므로 이 점을 유의해야 합니다. 따라서 상황에 맞는 적절한 방법을 선택하여 사용하는 것이 중요합니다. 때로는 방청 페인트를 선택할 때에는 시공과 유지보수가 더 중요한 요소라고 할 수 있습니다. 따라서, 방청 페인트를 선택할 때에는 시공과 유지보수를 종합적으로 고려해야 합니다. 따라서 적합한 방청 페인트를 선택하는 것이 중요합니다. 이를 통해 방청 페인트를 효과적으로 제거하여 아름다운 마감재를 유지할 수 있을 것입니다. 추가적으로, 방청 페인트를 사용할 경우에도 추가적인 관리가 필요하며, 이는 비용과 시간을 더 많이 필요로 합니다. 따라서 적합한 방청 페인트를 선택하여 사용하는 것이 좋습니다. 다만, 시공과 유지보수를 위해서는 전문가의 도움을 받는 것이 중요합니다. 따라서 정기적인 검토와 예산을 감안하여 적절한 대책을 마련하는 것이 중요합니다. 따라서 방청 페인트를 선택할 때에는 시공과 유지보수에 대한 신중한 고려가 필요합니다. 또한, 방청 페인트를 선택할 때에는 시공과 유지보수를 종합적으로 고려하는 것이 필요합니다. 따라서 이러한 측면을 주의하여 적절한 조치를 취하는 것이 필요합니다. 추가적인 측면에서는 전문가의 조언을 구하는 것이 필요합니다. 따라서 방


source_text :  도배지에 녹은 자국

In [ ]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

In [ ]:
submit = pd.read_csv('./bigdata/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

In [ ]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('./submit/dacon-v0_submit.csv', index=False)